# SAM Summarization MVP

This notebook will demonstrate the process of using a pre-trained BART model for summarization of conversations from the SAMSum data set.

### Data Loading/Imports

In [1]:
import pandas as pd
import numpy as np
import evaluate
import torch

from tqdm import tqdm
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer

train_df = pd.read_csv('samsum_csv_data/train.csv')
validate_df = pd.read_csv('samsum_csv_data/validation.csv')

In [2]:
poc_train_df = train_df.head(500)
poc_validate_df = validate_df.head(20)

# Convert pandas DataFrames to 🤗 HF Dataset objects (ref based)
hf_train_dataset = Dataset.from_pandas(poc_train_df)
hf_validate_dataset = Dataset.from_pandas(poc_validate_df)

MODEL_NAME = "facebook/bart-base"

print(f"\nLoading tokenizer for model: {MODEL_NAME}...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
print(f"Tokenizer vocabulary size: {len(tokenizer)}")


Loading tokenizer for model: facebook/bart-base...
Tokenizer vocabulary size: 50265


### Pre-processing/Tokenization

Code for data pre-processing. We'll set a relatively long max input size and create a warning if conversations are truncated.

In [3]:
# In token counts (effectively word)
MAX_INPUT_LENGTH = 1024 # Some dialogs will likely exceed this
MAX_TARGET_LENGTH = 128 # If our summary is trying to go longer than this it's wrong

def preprocess_function(examples):
    tokenized_inputs = tokenizer(
        examples["dialogue"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
        padding="max_length",
        return_overflowing_tokens=True,
        return_length=True
    )

    tokenized_labels = tokenizer(
        text_target=examples["summary"],
        max_length=MAX_TARGET_LENGTH,
        truncation=True,
        padding="max_length"
    )

    for i, length in enumerate(tokenized_inputs["length"]):
        if length == MAX_INPUT_LENGTH and "overflowing_tokens" in tokenized_inputs:
            overflow = tokenized_inputs["overflowing_tokens"][i] if i < len(tokenized_inputs["overflowing_tokens"]) else []
            overflow_text = tokenizer.decode(overflow, skip_special_tokens=True)
            print(f"⚠️ Truncated example {i}: {overflow_text[:100]}...")

    tokenized_inputs["labels"] = tokenized_labels["input_ids"]
    return tokenized_inputs


print(f"\nPreprocessing training/validation data (tokenizing and aligning lengths)...")
tokenized_hf_train_dataset = hf_train_dataset.map(
    preprocess_function,
    batched=True, # Process examples in batches for speed
    remove_columns=['id', 'dialogue', 'summary'] # Remove original text columns
)

tokenized_hf_validate_dataset = hf_validate_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=['id', 'dialogue', 'summary']
)


Preprocessing training/validation data (tokenizing and aligning lengths)...


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

### Training Parameters

We are optimizing these training parameters to be GPU-memory-friendly at the cost of performance hits in other areas. `fp16=True, gradient_checkpointing=True` were tried elsewhere and did not allow scaling to 1000 samples.

In [ ]:
tokenized_hf_train_dataset.set_format("torch")
tokenized_hf_validate_dataset.set_format("torch")

print(f"\nLoading BART model for sequence-to-sequence: {MODEL_NAME}...")
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
print("Model loaded successfully.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = model.to(device)

output_dir = "./bart_samsum_poc_results"
print(f"\nSetting up Training Arguments. Output directory: {output_dir}")
# Includes some default values like num_train_epochs for future clarity
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    warmup_steps=10,
    weight_decay=0.01,
    logging_steps=5,
    eval_strategy="steps",
    eval_steps=10,
    save_steps=10,
    report_to="none",
    gradient_accumulation_steps=2, # Memory issues
    load_best_model_at_end=True,
    metric_for_best_model="rouge1", # Look for higher ROUGE-1 scores 
    greater_is_better=True
)

print("\nTraining Arguments configured.")


Loading BART model for sequence-to-sequence: facebook/bart-base...
Model loaded successfully.
Using device: cuda

Setting up Training Arguments. Output directory: ./bart_samsum_poc_results

Training Arguments configured.


### Training Metrics and Initialization

As instructed, we'll use ROUGE scores - a measure of word overlap between our target "reference" summary and our model's generated summary.

In [5]:
print("\nLoading ROUGE metric...")
rouge_metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    predictions = predictions[0]
    predictions = np.argmax(predictions, axis=-1)

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Post-process for ROUGE: remove extra whitespace and newlines
    decoded_preds = ["\n".join(pred.strip().split()) for pred in decoded_preds]
    decoded_labels = ["\n".join(label.strip().split()) for label in decoded_labels]

    result = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    result = {k: round(v * 100, 4) for k, v in result.items()}
    return result

print("Metric computation function defined.")

print("\nInitializing the Trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_hf_train_dataset,
    eval_dataset=tokenized_hf_validate_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("Trainer initialized successfully. Ready for training.")


Loading ROUGE metric...
Metric computation function defined.

Initializing the Trainer...
Trainer initialized successfully. Ready for training.


/tmp/ipykernel_7257/288411429.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Training

Choose your own training montage music.

In [6]:
# Start the training process
print("\nStarting model training...")
train_result = trainer.train()
print("Training complete.")

# Save the trained model and tokenizer
trainer.save_model() # Saves the model and tokenizer to the output_dir specified in TrainingArguments
# For good measure, you can also save the tokenizer explicitly if desired (though save_model usually handles it)
# tokenizer.save_pretrained(training_args.output_dir)

# Save training metrics (optional, but good for tracking progress)
metrics = train_result.metrics
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)
trainer.save_state()

print(f"\nModel and tokenizer saved to: {training_args.output_dir}")
print("Training metrics logged and saved.")

# Optionally, you can also run a final evaluation on the validation set after training
print("\nRunning final evaluation on the validation set...")
eval_metrics = trainer.evaluate(eval_dataset=tokenized_hf_validate_dataset)
trainer.log_metrics("eval", eval_metrics)
trainer.save_metrics("eval", eval_metrics)
print("Final evaluation complete.")
print(eval_metrics) # Print the evaluation results


Starting model training...


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
10,11.204600,10.251661,42.117600,21.862000,39.190100,42.160200
20,8.172100,6.499501,28.307100,15.460400,27.330000,28.296700
30,5.337400,3.831983,53.516400,27.167200,51.186200,53.611200
40,3.582900,2.712469,55.076200,27.645000,51.537600,55.054500
50,2.632200,1.768104,56.127500,29.986300,54.374600,56.113300
60,1.783200,1.063573,59.022900,36.889200,57.750200,59.105900
70,1.191700,0.648764,56.315700,32.849800,54.723100,56.360100
80,0.965300,0.462819,58.052000,35.492800,56.658800,58.130400
90,0.830200,0.383688,58.753100,33.859600,57.085500,58.871200
100,0.535100,0.396347,53.088300,31.047400,51.459700,53.102200


/home/paul/miniconda3/envs/pytorch_env/lib/python3.10/site-packages/transformers/modeling_utils.py:3465: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


Training complete.
***** train metrics *****
  epoch                    =        3.0
  total_flos               =   851791GF
  train_loss               =     0.8301
  train_runtime            = 0:12:19.14
  train_samples_per_second =      2.029
  train_steps_per_second   =      1.015

Model and tokenizer saved to: ./bart_samsum_poc_results
Training metrics logged and saved.

Running final evaluation on the validation set...


***** eval metrics *****
  epoch                   =        3.0
  eval_loss               =     0.3291
  eval_rouge1             =    63.8728
  eval_rouge2             =    38.3221
  eval_rougeL             =    61.6562
  eval_rougeLsum          =    64.0096
  eval_runtime            = 0:00:01.83
  eval_samples_per_second =     10.899
  eval_steps_per_second   =      5.449
Final evaluation complete.
{'eval_loss': 0.32907551527023315, 'eval_rouge1': 63.8728, 'eval_rouge2': 38.3221, 'eval_rougeL': 61.6562, 'eval_rougeLsum': 64.0096, 'eval_runtime': 1.8351, 'eval_samples_per_second': 10.899, 'eval_steps_per_second': 5.449, 'epoch': 3.0}


### Evaluation

The model maintained its performance on conversations it hasn't already seen, indicating that overfitting us not a major concern presently. Let's take a look at some examples next.

In [7]:
# Define the output directory where your model was saved
output_dir = "./bart_samsum_poc_results"

# Load the fine-tuned model and tokenizer
print(f"\nLoading fine-tuned model and tokenizer from {output_dir}...")
device = "cpu"
print(f"Using device: {device}")

loaded_tokenizer = AutoTokenizer.from_pretrained(output_dir)
loaded_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir).to(device)

print("Model and tokenizer loaded successfully for inference.")

num_examples_to_show = 3

example_data = validate_df.sample(n=num_examples_to_show,
                                   random_state=42).to_dict('records')


print("\n--- Model Inference Examples ---")
for i, example in enumerate(example_data):
    dialogue = example['dialogue']
    reference_summary = example['summary']

    # Tokenize the input dialogue
    inputs = loaded_tokenizer(
        dialogue,
        return_tensors="pt", # Return PyTorch tensors
        max_length=MAX_INPUT_LENGTH, # Use the same max length as training
        truncation=True
    ).to(device) # Move inputs to the correct device (CPU/GPU)

    # Generate summary
    # Using parameters that typically work well for summarization
    summary_ids = loaded_model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_beams=4,        # Use beam search for better quality summaries
        max_length=MAX_TARGET_LENGTH, # Max length of generated summary
        min_length=30,      # Minimum length to encourage more detailed summaries
        early_stopping=True, # Stop generation when all beam hypotheses are complete
        length_penalty=2.0  # Encourage longer summaries (common for abstractive summarization)
    )

    # Decode the generated summary IDs back to text
    generated_summary = loaded_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"\n--- Example {i+1} ---")
    print(f"Original Dialogue:\n{dialogue}")
    print(f"\nReference Summary:\n{reference_summary}")
    print(f"\nGenerated Summary:\n{generated_summary}")
    print("-" * 30)


Loading fine-tuned model and tokenizer from ./bart_samsum_poc_results...
Using device: cpu
Model and tokenizer loaded successfully for inference.

--- Model Inference Examples ---

--- Example 1 ---
Original Dialogue:
Edd: wow, did you hear that they're transferring us to a different department?
Rose: whaaaaat :o
Rose: no! where'd you hear that?
Edd: well, it's quite official
Edd: Anderson just told us
Rose: and do you know what it changes for us?
Edd: they won't change the professors
Edd: but i know the paperwork will get trickier
Rose: and i guess that is a move that is supposed to make everything easier
Edd: yeah, guess so
Edd: they have a funny way of understanding 'to make things easier'

Reference Summary:
Rose and Edd will be transferred to a new department. Their professors will not change but paperwork will become more difficult.

Generated Summary:
Edd and Rose are transferred to a different department. They will change the professors, but the paperwork will get trickier. Th